In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("10-monkey-species"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'training', 'monkey_labels.txt', 'validation']


In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sys
import tensorflow as tf
import time
from tensorflow import keras

print(tf.__version__)

2.0.0


In [3]:
train_dir = "10-monkey-species/training/training"
valid_dir = "10-monkey-species/validation/validation"
label_file = "10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

True
True
True
['n0', 'n7', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']
['n0', 'n7', 'n9', 'n8', 'n6', 'n1', 'n4', 'n3', 'n2', 'n5']


In [4]:
labels = pd.read_csv(label_file, header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [5]:
# 定义图片大小
height = 224
width = 224
channels = 3
batch_size = 24
num_classes = 10

# 创建数据generator（针对图片的api）
# 里面的参数都是做数据增强的
train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input,   # 针对resnet50的预处理函数，用于做归一化到(-1, 1)
                                                             rotation_range = 40, # 旋转角度，随机旋转（-40， 40）度
                                                             width_shift_range = 0.2,   # 水平位移 
                                                             height_shift_range = 0.2,  # 垂直位移
                                                             shear_range = 0.2, # 剪切强度
                                                             zoom_range = 0.2, # 缩放强度
                                                             horizontal_flip = True, # 是否随机做水平翻转
                                                             fill_mode = 'nearest'  # 填充像素策略
                                                              )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'categorical' # 使用one_hot编码
                                                    )
valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_directory(valid_dir, 
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'categorical' # 使用one_hot编码
                                                    )

train_num = train_generator.samples  # 训练集数据数量
valid_num = train_generator.samples
print(train_num, valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 1098


In [6]:
# 从generator中取数据
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(24, 224, 224, 3) (24, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
(24, 224, 224, 3) (24, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0

## 使用resnet50模型做迁移学习

In [ ]:
resnet50_fine_tune = keras.models.Sequential()
resnet50_fine_tune.add(keras.applications.ResNet50(include_top=False,  # 去掉最后一层即输出层（resnet50解决的是1000分类，而我们的是10分类，所以要去掉重新定义）
                                                   pooling='avg',      # resnet50 倒数第二层的输出是一个卷积层的输出，是一个三维矩阵，因此需要：1.pooling掉两层 2.做flatten（pooling='None'）两种方法选一个
                                                   weights='imagenet'))# weights=['None', 'imagenet'], None: 从头开始训练, imagenet: 下载一个训练好的模型初始化网络结构
resnet50_fine_tune.add(keras.layers.Dense(num_classes, activation='softmax')) # 添加输出层
resnet50_fine_tune.layers[0].trainable = False   # 将网络结构第一层(resnet50这一层)fix住，不能训练，只训练输出层(这里resnet 50层的网络结构只当成一层)

resnet50_fine_tune.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
resnet50_fine_tune.summary()

 3244032/94765736 [>.............................] - ETA: 27:02:16

In [ ]:
epochs = 10
history = resnet50_fine_tune.fit_generator(train_generator, 
                                           steps_per_epoch = train_num // batch_size, 
                                           epochs=epochs, 
                                           validation_data=valid_generator,
                                           validation_steps=valid_num // batch_size
                                           )

In [ ]:
print(history.history.keys())

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()

plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2.5)

## 使resnet50后几层可以训练，前几层不能训练

In [ ]:
resnet50 = keras.applications.ResNet50(include_top=False,  # 去掉最后一层即输出层（resnet50解决的是1000分类，而我们的是10分类，所以要去掉重新定义）
                                       pooling='avg',      # resnet50 倒数第二层的输出是一个卷积层的输出，是一个三维矩阵，因此需要：1.pooling掉两层 2.做flatten（pooling='None'）两种方法选一个
                                       weights='imagenet'))# weights=['None', 'imagenet'], None: 从头开始训练, imagenet: 下载一个训练好的模型初始化网络结构

resnet50.summary()

In [ ]:
for layer in resnet50.layers[0: -5]:
    layer.trainable = False
resnet50_new = keras.layers.Sequential([
    resnet50,
    keras.layers.Dense(num_classes, activation='softmax')
])
resnet50_new.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
resnet50_new.summary()

In [ ]:
epochs = 10
history = resnet50_fine_tune.fit_generator(train_generator, 
                                           steps_per_epoch = train_num // batch_size, 
                                           epochs=epochs, 
                                           validation_data=valid_generator,
                                           validation_steps=valid_num // batch_size
                                           )
# 此时效果会变差一点，原因：
# 1.resnet后几层可以训练增加了可训练参数量，收敛需要更长的时间
# 2.resnet后几层已经训练好的，最后一次层是没有经过训练的，此时如果learning_rate较大，训练好的参数会被破坏掉，恢复到较好的状态需要较长的时间
# 因此需要减小learning_rate，增加迭代次数